# Optimization algorithms

## Gradient descent

| Type                        | Mini-batch size    | 
|-----------------------------|--------------------|
| Batch gradient descent      | $m_t = m$           |
| Stochastic gradient descent | $m_t = 1$           |
| Mini-batch gradient descent | $m_t \in (1, m)$     |

In [4]:
def update_parameters_with_gd(parameters, grads, learning_rate):
    L = len(parameters) // 2 # num of layers

    # update rule
    for l in range(1, L + 1):
        W = parameters["W" + str(l)]
        dW = grads["dW" + str(l)]
        b = parameters["b" + str(l)]
        db = grads["db" + str(l)]
        parameters["W" + str(l)] = W - learning_rate * dW
        parameters["b" + str(l)] = b - learning_rate * db
    
    return parameters

### Batch gradient descent

In [5]:
def batch_gradient_descent(X, Y, layers_dims, num_iterations, learning_rate):
    m = X.shape[1]  # num of train examples
    parameters = initialize_parameters(layers_dims)

    for i in range(num_iterations):
        a, caches = forward_propagation(X, parameters)
        cost_total = compute_cost(a, Y)
        grads = backward_propagation(a, caches, parameters)
        parameters = update_parameters_with_gd(parameters, grads, learning_rate)
        cost_avg = cost_total / m
        
    return parameters

### Stochastic gradient descent

Implementing SGD needs 3 loops:
1. Over the number of iterations
2. Over the $m$ training examples
3. Over the layers—to update all parameters, from $(W^{[1]},b^{[1]})$ to $(W^{[L]},b^{[L]})$

In [6]:
def stochastic_gradient_descent(X, Y, layers_dims, num_iterations, learning_rate):
    m = X.shape[1]  # num of train examples
    parameters = initialize_parameters(layers_dims)

    for i in range(num_iterations):
        cost_total = 0
        for j in range(m):
            a, caches = forward_propagation(X[:, j:j+1], parameters)
            cost_total += compute_cost(a, Y[:, j:j+1])
            grads = backward_propagation(a, caches, parameters)
            parameters = update_parameters_with_gd(parameters, grads, learning_rate)
        cost_avg = cost_total / m
    
    return parameters

### Mini-batch gradient descent

In mini-batch gradient descent there are multiple gradient steps—once per mini-batch. 

If there's 5000 mini-batches, there's 5000 updates per epoch.

If train set is small $(m \le 2000)$: use batch GD. Otherwise choose a mini-batch size.

Use powers of 2 for typical mini-batch sizes: $64, 128, 256, 512, 1024, \ldots$ due to hardware memory alignment.

There are two steps in building mini-batches: shuffling and partioning.

1. Shuffling
	- Create a shuffled version of the train set $(X, Y)$.
	- Each column of $X$ an $Y$ corresponds to a single train example.
	- Shuffle synchronously—the $i^\text{th}$ column $X$ stays paired with the $i^\text{th}$ column of $Y$ after shuffling.
	- Shuffling ensures that examples are split randomly into different mini-batches.
2. Partitioning
	- Divide the shuffled train set into mini-batches of size $m_t$.
	- If the num of train examples is not divisible by $m_t$, the final mini-batch will have fewer examples. This is normal.

In [8]:
def random_mini_batches(X, Y, mini_batch_size = 64):
    
    m = X.shape[1] # num of train examples
    mini_batches = []
        
    # shuffling
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1, m))
    
    # partition
    num_complete_minibatches = math.floor(m / mini_batch_size) # num of mini batches
    for k in range(0, num_complete_minibatches):
        start = k * mini_batch_size
        end = (k+1) * mini_batch_size
        mini_batch_X = shuffled_X[:, start:end]
        mini_batch_Y = shuffled_Y[:, start:end]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # end case when m is not divisisible by m_t
    if m % mini_batch_size != 0:
        rem_start = num_complete_minibatches * mini_batch_size
        rem_end = m
        mini_batch_X = shuffled_X[:, rem_start:rem_end]
        mini_batch_Y = shuffled_Y[:, rem_start:rem_end]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

Mini-batch gradient descent procedure:
1. Forward propagation on $X^{t}$
	1. $Z^{[1]} = W^{[1]} X^{\{t\}} + b^{[1]}$
	2. $A^{[1]} = g^{[1]}(Z^{[1]})$
	3. $\ldots$
	4. $Z^{[1]} = W^{[1]} A^{[L-1]} + b^{[L]}$
	5. $A^{[L]} = g^{[L]}(Z^{[L]})$
2. Compute cost for mini-batch $\displaystyle J^{\{t\}} = \frac{1}{m_t} \sum_{i=1}^{1000} \mathcal{L}(\hat y^{(i)}, y^{(i)}) + \frac{\lambda}{2m} + \sum_{l} \vert \vert W^{[l]} \vert \vert_{F}^2$
3. Backward propagation by computing gradients w.r.t. $J^{\{t\}}$ using ($X^{\{t\}}, Y^{\{t\}})$
	1. $dZ^{[L]} = A^{[L]} - Y^{\{t\}}$
	2. $dW^{[L]} = \frac{1}{m_t} dZ^{[L]}(A^{[L-1]})^\intercal + \frac{\lambda}{m} W^{[L]}$
	3. $db^{[L]} = \frac{1}{m_t} \sum dZ^{[L]}$
	4. $\ldots$
	5. $dA^{[1]} = (W^{[2]})^\intercal \cdot dZ^{[2]}$
	6. $dZ^{[1]} = dA^{[1]} \ast g^{[1]'}(Z^{[1]})$
	7.  $dW^{[1]} = \frac{1}{m_t} dZ^{[1]} \cdot X^\intercal + \frac{\lambda}{m} W^{[1]}$
	8. $db^{[1]} = \frac{1}{m_t} \sum dZ^{[1]}$
4. Update the parameters
	1. $W^{[l]} := W^{[l]} - \alpha \cdot dW^{[l]}$
	2. $b^{[l]} := b^{[l]} - \alpha \cdot db^{[l]}$


In [9]:
def mini_batch_gradient_descent(X, Y, layers_dims, num_epochs, mini_batch_size, learning_rate):
    parameters = initialize_parameters(layers_dims)

    for epoch in range(num_epochs):
        # create shuffled mini-batches
        mini_batches = random_mini_batches(X, Y, mini_batch_size)

        # loop over each mini-batch
        for mini_batch in mini_batches:
            (mini_batch_X, mini_batch_Y) = mini_batch

            a, caches = forward_propagation(mini_batch_X, parameters)
            cost_total = compute_cost(a, mini_batch_Y)
            grads = backward_propagation(a, caches, parameters)
            parameters = update_parameters_with_gd(parameters, grads, learning_rate)

        print(f'Epoch {epoch}: Cost = {cost_total / mini_batch_size}')

    return parameters

## Momentum

Problem with standard gradient descent:
1. When optimizing $J$ with elongated contours, the algorithm may oscillate back and forth across the narrow dimension (e.g., vertical axis).
2. This forces the use of a small learning rate $\alpha$, since a large $\alpha$ might cause overshooting and end up diverging.
3. Convergence slows due to small $\alpha$.

What do we want?
- Slow learning in the vertical direction to prevent oscillations.
- Fast learning in the horizontal direction to quickly reach the minimum.

The solution? Use momentum.
- Momentum smooths the optimization path by averaging gradients over time.
- Averaging causes vertical oscillations to cancel out.
- Averaging causes horizontal direction to accumulate.

In [10]:
def initialize_velocity(parameters):
    L = len(parameters) // 2 # num of layers
    v = {}
    
    for l in range(1, L + 1):
        v["dW" + str(l)] = np.zeros(parameters["W" + str(l)].shape)
        v["db" + str(l)] = np.zeros(parameters["b" + str(l)].shape)
        
    return v

In [15]:
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):
    """
    Arguments:
    parameters: dict containing your params:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads: dict containing your grads for each params:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v: dict containing the current velocity:
                    v['dW' + str(l)] = ...
                    v['db' + str(l)] = ...
    beta: the momentum hyperparameter, scalar
    learning_rate: the learning rate, scalar
    
    Returns:
    parameters: dict containing updated params 
    v: dictionary containing updated velocities
    """
    L = len(parameters) // 2 # num of layers
    
    # momentum update for each parameter
    for l in range(1, L + 1):
        # compute velocities
        v["dW" + str(l)] = beta * v["dW" + str(l)] + (1 - beta) * grads["dW" + str(l)]
        v["db" + str(l)] = beta * v["db" + str(l)] + (1 - beta) * grads["db" + str(l)]

        # update parameters
        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate * v["dW" + str(l)]
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate * v["db" + str(l)]
        
    return parameters, v

## Adam

Adam (_Adaptive Moment Estimation_) merges momentum and RMSprop.

Background:
- Momentum uses EWAs of gradients to smooth updates.
- RMSprop uses EWAs of squared gradients to scale updates adaptively.

$V_{dW}$ is the first moment (the mean) which guides the direction of the updates. \
$S_{dW}$ is the second moment (the uncentered variance) which adjusts the step size for each parameter update.

Initialize all moment vectors at $t = 0$:
- $v_{dW} = 0$
- $v_{db} = 0$
- $S_{dW} = 0$
- $S_{db} = 0$

During iteration $t$:
1. Compute the gradients $dW$, $db$ on the current mini-batch.
2. Momentum update (first moment)
	- $v_{dW} = \beta_1 \cdot v_{dW} + (1-\beta_1) \cdot dW$
	- $v_{db} = \beta_1 \cdot v_{db} + (1-\beta_1) \cdot db$
3. RMSprop update (second moment)
	- $S_{dW} = \beta_2 \cdot S_{dW} + (1-\beta_2) \cdot dW^2$
	- $S_{db} = \beta_2 \cdot S_{db} + (1-\beta_2) \cdot db^2$
4. Bias correction due to the bias to $0$ caused by initialization
	- $\displaystyle \hat v_{dW} = \frac{v_{dW}}{1-\beta_1^t}$
	- $\displaystyle \hat v_{db} = \frac{v_{db}}{1-\beta_1^t}$
	- $\displaystyle \hat S_{dW} = \frac{S_{dW}}{1-\beta_2^t}$
	- $\displaystyle \hat S_{db} = \frac{S_{db}}{1-\beta_2^t}$
5. Parameter update
	- $\displaystyle W := W - \alpha \frac{\hat v_{dW}}{\sqrt{\hat S_{dW}} + \varepsilon}$
	- $\displaystyle b := b - \alpha \frac{\hat v_{db}}{\sqrt{\hat S_{db}} + \varepsilon}$

| Hyperparameter | Symbol | Typical value |
|----------------|--------|---------------|
| Learning rate | $\alpha$ | needs to be tuned |
| Momentum decay rate (first moment) | $\beta_1$ | 0.9 |
| RMSprop decay rate (second moment) | $\beta_2$ | 0.999 |
| Numerical stability constant | $\varepsilon$ | $10^{-8}$ |

Both 0.9 and 0.999 are values proposed by the authors (Kingma et. al.).


In [13]:
def initialize_adam(parameters):    
    L = len(parameters) // 2 # num of layers
    v = {}
    s = {}
    
    for l in range(1, L + 1):
        v["dW" + str(l)] = np.zeros(parameters["W" + str(l)].shape)
        v["db" + str(l)] = np.zeros(parameters["b" + str(l)].shape)
        s["dW" + str(l)] = np.zeros(parameters["W" + str(l)].shape)
        s["db" + str(l)] = np.zeros(parameters["b" + str(l)].shape)
    
    return v, s

In [18]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Args:
    parameters: dict containing params:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads: dict containing grads for each param:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v: Adam variable, moving ave of the first grad, a dict
    s: Adam variable, moving ave of the squared grad, a dict
    t: Adam variable, counts the num of steps taken for bias correction
    learning_rate: learning rate, scalar
    beta1: exponential decay hyperparameter for the first moment estimates
    beta2: exponential decay hyperparameter for the second moment estimates
    epsilon: hyperparameter to prevent division by zero in the Adam update, a small scalar

    Returns:
    parameters: dict containing updated params
    v: updated Adam variable, moving ave of the first grad, a dict
    s: updated Adam variable, moving ave of the squared grad, a dict
    v_corrected: dict containing bias-corrected first moment estimates
    s_corrected: dict containing bias-corrected second moment estimates
    """
    
    L = len(parameters) // 2 # num of layers
    v_corrected = {} # init first moment estimate
    s_corrected = {} # init second moment estimate
    
    for l in range(1, L + 1):
        # moving ave of grads
        v["dW" + str(l)] = beta1 * v["dW" + str(l)] + (1 - beta1) * grads["dW" + str(l)]
        v["db" + str(l)] = beta1 * v["db" + str(l)] + (1 - beta1) * grads["db" + str(l)]

        # bias-corrected first moment estimate
        v_corrected["dW" + str(l)] = v["dW" + str(l)] / (1 - beta1 ** t)
        v_corrected["db" + str(l)] = v["db" + str(l)] / (1 - beta1 ** t)

        # moving ave of the squared grads
        s["dW" + str(l)] = beta2 * s["dW" + str(l)] + (1 - beta2) * (grads["dW" + str(l)])**2
        s["db" + str(l)] = beta2 * s["db" + str(l)] + (1 - beta2) * (grads["db" + str(l)])**2
        
        # bias-corrected second raw moment estimate
        s_corrected["dW" + str(l)] = s["dW" + str(l)] / (1 - beta2 ** t)
        s_corrected["db" + str(l)] = s["db" + str(l)] / (1 - beta2 ** t)
        
        # update params
        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate * v_corrected["dW" + str(l)] / (np.sqrt(s_corrected["dW" + str(l)]) + epsilon)
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate * v_corrected["db" + str(l)] / (np.sqrt(s_corrected["db" + str(l)]) + epsilon)

    return parameters, v, s, v_corrected, s_corrected

## Learning rate decay

Learning rate decay can speed up a learning algorithm by slowly reducing learning rate over time.

Exponential learning rate decay:
$$\alpha = \frac{1}{1 + \text{decay rate} \times \text{epoch number}} \alpha_{0}$$
- $\text{decay factor} < 1$
- $\alpha$ decreases exponentially fast.

Inverse learning rate decay:
$$\alpha = \frac{k \cdot a_0}{\sqrt{\text{epoch num}}}$$
- $k$ is a constant.

Mini-batch learning rate decay:
$$\alpha = \frac{k \cdot a_0}{\sqrt{t}}$$
- $t$ is the mini-batch number.

In [28]:
def update_lr(learning_rate0, epoch_num, decay_rate):
    denominator = 1 + decay_rate * epoch_num
    learning_rate = (1 / denominator) * learning_rate0
    
    return learning_rate

The new learning rate using exponential weight decay with fixed interval scheduling.

$$\alpha = \frac{1}{1 + \text{decay rate} \times \lfloor\frac{\text{epoch num}}{\text{time interval}}\rfloor} \alpha_{0}$$

In [29]:
def schedule_lr_decay(learning_rate0, epoch_num, decay_rate, time_interval=1000):
    denominator = 1 + decay_rate * np.floor(epoch_num/time_interval)
    learning_rate = (1 / denominator) * learning_rate0
    
    return learning_rate

## Model with different optimization algorithms and learning rate decay

3-layer neural network model which can be run in different optimizer modes.

In [21]:
def model(X, Y, layers_dims, optimizer, learning_rate=0.0007, 
          mini_batch_size=64, beta=0.9, beta1=0.9, beta2=0.999,
          epsilon=1e-8, num_epochs=5000, print_cost=True, decay=None, decay_rate=1):
    
    L = len(layers_dims) # num of layers
    costs = []                       
    t = 0 # counter required for Adam update
    m = X.shape[1] # num of train examples
    lr_rates = []
    learning_rate0 = learning_rate   # original learning rate
    
    # init params
    parameters = initialize_parameters(layers_dims)

    # init optimizer
    if optimizer == "gd":
        pass
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    # optimization loop
    for i in range(num_epochs):
        
        # create random minibatches
        minibatches = random_mini_batches(X, Y, mini_batch_size)
        cost_total = 0
        
        for minibatch_X, minibatch_Y in minibatches:
            a3, caches = forward_propagation(minibatch_X, parameters) # forward prop
            cost_total += compute_cost(a3, minibatch_Y) # compute cost and add to total
            grads = backward_propagation(minibatch_X, minibatch_Y, caches) # back prop

            # update params
            if optimizer == "gd":
                parameters = update_parameters_with_gd(parameters, grads, learning_rate)
            elif optimizer == "momentum":
                parameters, v = update_parameters_with_momentum(parameters, grads, v, beta, learning_rate)
            elif optimizer == "adam":
                t += 1  # Adam counter
                parameters, v, s, _, _ = update_parameters_with_adam(
                    parameters, grads, v, s, t, learning_rate, beta1, beta2, epsilon)

        cost_avg = cost_total / m

        # appy learning rate decay
        if decay:
            learning_rate = decay(learning_rate0, i, decay_rate)

        # print cost every 1000 epochs
        if print_cost and i % 1000 == 0:
            print(f"Cost after epoch {i}: {cost_avg:.6f}")
            if decay:
                print(f"Learning rate after epoch {i}: {learning_rate:.6f}")
        if print_cost and i % 100 == 0:
            costs.append(cost_avg)
                
    # plot cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('epochs (per 100)')
    plt.title(f"Learning rate = {learning_rate}")
    plt.show()

    return parameters
